In [2]:
# Analyze simulation data

import pandas as pd
import ast
import json
import teams.teams_helpers as teams_helpers
import params_team as params
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import math
import os
from termcolor import colored
import warnings
import textwrap
import pickle
from ast import literal_eval
import copy
from collections import Counter
plt.rcParams['figure.figsize'] = [15, 10]

import teams.teams_helpers as team_helpers
import params_team as params
import policy_summarization.BEC_helpers as BEC_helpers
# import simulation.sim_helpers as sim_helpers
from simple_rl.planning import ValueIteration


# from SALib.analyze import sobol
from pingouin import partial_corr


import pandas as pd
import ast
import json
import teams.teams_helpers as teams_helpers
import params_team as params
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import math
import os
from termcolor import colored
import warnings
import textwrap
import pickle
from ast import literal_eval
import copy

plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 14

# Set the seaborn color palette to colorblind
sns.set_palette("colorblind")

# Get the colorblind palette with the desired number of colors
colorblind_palette = sns.color_palette("colorblind", 6)

# Set the default color cycle for Matplotlib using rcParams
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=colorblind_palette)
matplotlib.use('TkAgg')


import teams.teams_helpers as team_helpers
import params_team as params
# import simulation.sim_helpers as sim_helpers
from teams import particle_filter_team as pf_team
import policy_summarization.BEC_visualization as BEC_viz
import policy_summarization.BEC_helpers as BEC_helpers

import sage.geometry.polyhedron.base as Polyhedron
import sage.all
from statsmodels.graphics.factorplots import interaction_plot
from pandas.api.types import CategoricalDtype


# from SALib.analyze import sobol
from pingouin import partial_corr
import teams.utils_teams as utils_teams
import random

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn import metrics
from scipy.stats import bartlett, levene

from policy_summarization import computational_geometry as cg

import ast
from statsmodels.stats.anova import anova_lm
import dill as pickle

warnings.simplefilter(action='ignore', category=FutureWarning)


/home/sureshkj/anaconda3/envs/aug_taxi/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [3]:
with open('models/skateboard2/team_BEC_constraints.pickle', 'rb') as f:
    team_constraints = pickle.load(f)

In [1]:
# team_constraints

In [8]:
with open('models/skateboard2/team_base_constraints.pickle', 'rb') as f:
    team_base_constraints = pickle.load(f)

In [2]:
# team_base_constraints

In [5]:
with open('models/skateboard2/base_constraints.pickle', 'rb') as f:
    base_constraints = pickle.load(f)

In [6]:
with open('models/skateboard2/BEC_constraints.pickle', 'rb') as f:
    BEC_constraints = pickle.load(f)

In [3]:
# BEC_constraints

In [4]:
# check minimum constraints

# running_constraints = [np.array([[0,  0,  -1]]), np.array([[-1,  0,  2]])]
# running_constraints = [np.array([[-1,  0,  2]]), np.array([[ 0, -1, -4]]), np.array([[ 1,  0, -4]]), np.array([[0, 1, 2]])]
running_constraints = [np.array([[0,  0,  -1]]), np.array([[-1,  0,  0]]), np.array([[-1,  0,  2]])]


min_constraints = BEC_helpers.remove_redundant_constraints(running_constraints, params.weights['val'], params.step_cost_flag)

print(min_constraints)

print(BEC_helpers.calc_solid_angles([min_constraints]))

[array([[-1,  0,  2]]), array([[ 0,  0, -1]])]
[0.9272952180016123]


In [1]:
teacher_uf = 0.75

team_prior, particles_team_teacher = team_helpers.sample_team_pf(params.team_size, params.BEC['n_particles'], params.weights['val'], params.step_cost_flag, teacher_learning_factor=[teacher_uf,teacher_uf,teacher_uf], team_prior = params.team_prior, model_type = params.teacher_update_model_type)

In [2]:
teacher_uf = 0.75

particles_demo = copy.deepcopy(particles_team_teacher['p1'])

running_constraints = [np.array([[-1,  0,  0]]), np.array([[-1,  0,  2]])]

particles_demo.update(running_constraints, teacher_uf, viz_flag=True, model_type = params.teacher_update_model_type)

In [3]:
def analyze_demo_strategies(path, files, file_prefix_list, runs_to_exclude_list = [], vars_filename_prefix = ''):

    try:
        with open(path + '/' + vars_filename_prefix + '_demo_strategies_subset_N15_checked2.pickle', 'rb') as f:
            run_data_df_subset = pickle.load(f)
        print('Opening file: ', vars_filename_prefix + '_demo_strategies_subset_N15_checked.pickle')

    except:
        run_data_list = []
        run_data_learning_incomplete_list = []
        analysis_run_id = 1

        with open(path + '/sim_study_no_dup_N6_sample_cluster_weights_trial_data_subset_N15_checked.pickle', 'rb') as f:
            trial_data = pickle.load(f)

        unique_runs = trial_data['analysis_run_id'].unique()


        for file in files:
            # check if file is a valid pickle file
            
            for file_prefix in file_prefix_list:
                if file_prefix in file and '.pickle' in file and '.png' not in file and 'counterfactual' not in file:
                    run_file_flag = True
                    break
                else:
                    run_file_flag = False


            # check if file needs to be excluded
            for runs_to_exclude in runs_to_exclude_list:
                if runs_to_exclude in file:
                    run_file_flag = False
                    break
            

            if run_file_flag:
                with open(path + '/sim_data/' + file, 'rb') as f:
                    sim_vars = pickle.load(f)
                print('Opening file: ', file)
        
                study_id = sim_vars['study_id'][0]
                sim_run_id = sim_vars['run_no'][0]

                # check if learning was completed based on sim status flag
                if sim_vars['sim_status'].iloc[-1] == 'Teaching complete':
                    learning_complete = True
                else:
                    learning_complete = False

                # read counterfactuals sampled models data
                folder = file.strip('.pickle')
                demo_file_name = path + '/ind_sim_trials/' + folder + '/demo_gen_log.txt'

                try:
                    with open(path + '/ind_sim_trials/' + folder + '_counterfactuals.pickle', 'rb') as f:
                        counterfactuals = pickle.load(f)
                    print('Already compiled counterfactuals data. Reading from file. ')
                except:
                    if 'missing' in file:
                        counterfactuals = read_demo_gen_log(demo_file_name, new_flag=True)
                    else:
                        counterfactuals = read_demo_gen_log(demo_file_name, new_flag=True)

                ############## compile data
                study_id = sim_vars['study_id'].iloc[0]
                run_id = sim_vars['run_no'].iloc[0]
                team_composition = str(sim_vars['team_composition'].iloc[0])
                demo_strategy = str(sim_vars['demo_strategy'].iloc[0])
                sim_status = str(sim_vars['sim_status'].iloc[-1])
                min_BEC_constraints = sim_vars['min_BEC_constraints'].iloc[-1]
                running_BEC_constraints = copy.deepcopy(params.prior)
                print('Prior constraints: ', params.prior)
                # filter extra sims in same file
                kc_id_list = sim_vars['knowledge_comp_id']
                run_start_id = []
                for id in range(len(kc_id_list)-1):
                    if kc_id_list[id+1] - kc_id_list[id] < 0:
                        print(colored('Duplicate trials found for file: ' + file + '. Run id: ' + str(sim_run_id), 'red' ))
                        run_start_id = [id+1]
                        break
                
                if len(run_start_id) != 0:
                    sim_vars = sim_vars.iloc[run_start_id[0]:].reset_index(drop=True)
                ##################
                    
                for i, int_data in sim_vars.iterrows():
                    print('interaction: ', i+1)
                    run_data_dict = {'study_id': study_id, 'run_id': run_id, 'analysis_run_id': analysis_run_id, 'team_composition': team_composition, 'demo_strategy': demo_strategy, 'sim_status': sim_status, \
                                        'int_id': int_data['loop_count']}
                    
                    run_data_dict['kc_id'] = int_data['knowledge_comp_id']
                    run_data_dict['min_KC_constraints'] = int_data['min_KC_constraints']
                    print('Previous running demo constraints: ', running_BEC_constraints)
                    running_BEC_constraints.extend(int_data['min_KC_constraints'])
                    print('min KC constraints: ', int_data['min_KC_constraints'])

                    run_data_dict['KC_constraints_area'] = BEC_helpers.calc_solid_angles([int_data['min_KC_constraints']])[0]
                    run_data_dict['BEC_constraints_area'] = BEC_helpers.calc_solid_angles([int_data['min_BEC_constraints']])[0]
                    print('Demo constraints: ', running_BEC_constraints)
                    running_BEC_constraints = BEC_helpers.remove_redundant_constraints(running_BEC_constraints, params.weights['val'], params.step_cost_flag)
                    print('Min demo constraints: ', running_BEC_constraints)
                    run_data_dict['running_BEC_constraints'] = copy.deepcopy(running_BEC_constraints)
                    run_data_dict['running_BEC_constraints_area'] = BEC_helpers.calc_solid_angles([running_BEC_constraints])[0]
        
                    if i > 0:
                        run_data_dict['delta_BEC_constraints_area'] = running_BEC_constraints_area_prev - run_data_dict['running_BEC_constraints_area']
                        run_data_dict['scale_reduce_BEC_constraints_area'] = running_BEC_constraints_area_prev/run_data_dict['running_BEC_constraints_area'] # area reduction (information gain) # prior BEC area (step cost is negative)
                    else:
                        run_data_dict['delta_BEC_constraints_area'] = 2*np.pi - run_data_dict['running_BEC_constraints_area'] # area reduction (information gain) # prior BEC area (step cost is negative)
                        run_data_dict['scale_reduce_BEC_constraints_area'] = 2*np.pi/run_data_dict['running_BEC_constraints_area'] # area reduction (information gain) # prior BEC area (step cost is negative)
                    
                    
                    BEC_knowledge_team = 0
                    for member in int_data['BEC_knowledge_level']:
                        run_data_dict['BEC_knowledge_level_' + member] = int_data['BEC_knowledge_level'][member]
                        if 'p' in member:
                            # print(int_data['BEC_knowledge_level'][member])
                            mem_know = [float(x) for x in int_data['BEC_knowledge_level'][member]]
                            # print(mem_know)
                            BEC_knowledge_team += np.sum(mem_know)

                    run_data_dict['BEC_knowledge_team'] = BEC_knowledge_team/3

                    # find sampled counterfactuals
                    summary_count = int_data['summary_count']
                    counterfactual_models_list = counterfactuals[counterfactuals['length_of_summary']==summary_count]['sample_human_models'].reset_index(drop=True)
                    counterfactual_models = counterfactual_models_list.iloc[0]
                    counterfactual_models = [np.array(x).astype(float) for x in counterfactual_models]

                    # calculate how close the sampled points are
                    counterfactual_models_latllong = cg.cart2latlong(counterfactual_models)
                    pairwise = metrics.pairwise.haversine_distances(counterfactual_models_latllong)
                    # print(pairwise)
                    run_data_dict['pairwise_dist_mean'] = np.mean(pairwise)
                    run_data_dict['pairwise_dist_std'] = np.std(pairwise)
                    run_data_dict['sampled_models_cluster_idxs'] = counterfactuals[counterfactuals['length_of_summary']==summary_count]['sampled_models_cluster_idxs'].iloc[0]
                    run_data_dict['cluster_weights'] = counterfactuals[counterfactuals['length_of_summary']==summary_count]['cluster_weights'].iloc[0]

                    # update end of loop vars
                    running_BEC_constraints_area_prev = run_data_dict['running_BEC_constraints_area']

                    # print(run_data_dict)
                    if learning_complete:
                        run_data_list.append(run_data_dict)
                    else:
                        run_data_learning_incomplete_list.append(run_data_dict)
                    

                    with open(path + '/ind_sim_trials/' + folder + '_counterfactuals.pickle', 'wb') as f:
                        pickle.dump(counterfactuals, f)
                    counterfactuals.to_csv(path + '/ind_sim_trials/' + folder + '_counterfactuals.csv')

                ####################
                if learning_complete:
                    analysis_run_id += 1
        
        ################ end of reading files ##########
        
        run_data_df = pd.DataFrame(run_data_list)
        run_data_incomplete_df = pd.DataFrame(run_data_learning_incomplete_list)

        with open(path + '/' + vars_filename_prefix + '_demo_strategies_N15_checked.pickle', 'wb') as f:
            pickle.dump(run_data_df, f)

        run_data_df.to_csv(path + '/' + vars_filename_prefix + '_demo_strategies_N15_checked.csv')


        with open(path + '/' + vars_filename_prefix + '_demo_strategies_learning_incomplete.pickle', 'wb') as f:
            pickle.dump(run_data_incomplete_df, f)
        
        run_data_incomplete_df.to_csv(path + '/' + vars_filename_prefix + '_demo_strategies_learning_incomplete.csv')

        index_to_choose = run_data_df['analysis_run_id'].isin(unique_runs).index
        run_data_df_subset = run_data_df.iloc[index_to_choose]

        with open(path + '/' + vars_filename_prefix + '_demo_strategies_subset_N15_checked.pickle', 'wb') as f:
            pickle.dump(run_data_df_subset, f)

        run_data_df_subset.to_csv(path + '/' + vars_filename_prefix + '_demo_strategies_subset_N15_checked.csv')


In [3]:
# print('Prior: ', params.prior)
# path = 'models/augmented_taxi2'
# files = ['03_08_sim_study_no_dup_N6_sample_cluster_weights_study_1_run_7.pickle']
# file_prefix_list = ['03_08_sim_study_no_dup_N6_sample_cluster_weights']
# vars_filename = 'test'
# analyze_demo_strategies(path, files, file_prefix_list)

In [10]:
model = 2403
env = 40
filename = 'models/skateboard2/counterfactual_data_precomputed/model' + str(model) + '/cf_data_env' + str(env).zfill(5) + '.pickle'

print('Opening ', filename)
with open(filename, 'rb') as f:
    data = pickle.load(f)
    
print(data)

Opening  models/skateboard2/counterfactual_data_precomputed/model2403/cf_data_env00040.pickle
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [ ]:
# Check for valid reward weights for each domain
N = 500

mdp_class_list = ['augmented_taxi2', 'skateboard2']

env_list = np.arange(64)

valid_env_points = {}

for mdp_class in mdp_class_list:
    
    valid_env_points[mdp_class] = []

    params.mdp_class = mdp_class
    
    with open('models/' + params.data_loc['BEC'] + '/team_base_constraints.pickle', 'rb') as f:
        policy_constraints, min_subset_constraints_record, env_record, traj_record, traj_features_record, reward_record, mdp_features_record, consistent_state_count = pickle.load(f)


    N_points = BEC_helpers.sample_human_models_uniform([], N)
    
    point_idx = 0
    for point in N_points:
        print('Point idx: ', point_idx)
        
        for env_idx in env_list:
            
#             filename = 'models/' + params.mdp_class + '/gt_policies/wt_vi_traj_params_env' + str(env_idx).zfill(5) + '.pickle'
#             with open(filename, 'rb') as f:
#                 wt_vi_traj_env = pickle.load(f)
                        
            mdp = wt_vi_traj_env[0][1].mdp
#             mdp.set_init_state(opt_traj[0][0])
            mdp.weights = point
            vi_human = ValueIteration(mdp, sample_rate=1, max_iterations=100)
            vi_human.run_vi()
            
            
            if vi_human.stabilized:
#                 print(colored('\033[92mValid point '+ str(point) + 'for env ' + str(env_idx), 'green'))
                valid_env_points[mdp_class].append((env_idx, point_idx, point))
#             else:
#                 print(colored('\033[91mInvalid point '+ str(point) + 'for env ' + str(env_idx), 'red'))
                            
        ########  end of env loop
        
        point_idx += 1

        
        
with open('models/' + params.mdp_class + '/valid_env_points.pickle', 'wb') as f:
    pickle.dump(valid_env_points, f)
        
print(len(valid_env_points))
        
    

num_points:  500
Point idx:  0
Point idx:  1
Point idx:  2
Point idx:  3
Point idx:  4
Point idx:  5
Point idx:  6
Point idx:  7
Point idx:  8
Point idx:  9
Point idx:  10
Point idx:  11
Point idx:  12
Point idx:  13
Point idx:  14
Point idx:  15
Point idx:  16
Point idx:  17
Point idx:  18
Point idx:  19
Point idx:  20
Point idx:  21
Point idx:  22
Point idx:  23
Point idx:  24
Point idx:  25
Point idx:  26
Point idx:  27
Point idx:  28
Point idx:  29
Point idx:  30
Point idx:  31
Point idx:  32
Point idx:  33
Point idx:  34
Point idx:  35
Point idx:  36
Point idx:  37
Point idx:  38
Point idx:  39
Point idx:  40
Point idx:  41
Point idx:  42
Point idx:  43
Point idx:  44
Point idx:  45
Point idx:  46
Point idx:  47
Point idx:  48
Point idx:  49
Point idx:  50
Point idx:  51
Point idx:  52
Point idx:  53
Point idx:  54
Point idx:  55
Point idx:  56
Point idx:  57
Point idx:  58
Point idx:  59
Point idx:  60
Point idx:  61
Point idx:  62
Point idx:  63
Point idx:  64
Point idx:  65
Poi

Point idx:  19
Point idx:  20
Point idx:  21
Point idx:  22
Point idx:  23
Point idx:  24
Point idx:  25
Point idx:  26
Point idx:  27
Point idx:  28
Point idx:  29
Point idx:  30
Point idx:  31
Point idx:  32
Point idx:  33
Point idx:  34
Point idx:  35
Point idx:  36
Point idx:  37
Point idx:  38
Point idx:  39
Point idx:  40
Point idx:  41
Point idx:  42
Point idx:  43
Point idx:  44
Point idx:  45
Point idx:  46
Point idx:  47
Point idx:  48
Point idx:  49
Point idx:  50
Point idx:  51
Point idx:  52
Point idx:  53
Point idx:  54
Point idx:  55
Point idx:  56
Point idx:  57
Point idx:  58
Point idx:  59
Point idx:  60
Point idx:  61
Point idx:  62
Point idx:  63
Point idx:  64
Point idx:  65
Point idx:  66
Point idx:  67
Point idx:  68
Point idx:  69
Point idx:  70
Point idx:  71
Point idx:  72
Point idx:  73
Point idx:  74
Point idx:  75
Point idx:  76
Point idx:  77
Point idx:  78
Point idx:  79
Point idx:  80
Point idx:  81
Point idx:  82
Point idx:  83
Point idx:  84
Point idx: